In [1]:
import requests
import time
import os, json
import csv
import concurrent.futures

In [3]:
# url = "https://refract.fosfor.com/magiccodermodel/6188a3e2-bc43-444e-bfaa-8159fca71b3a/score"
url = "http://vllm-serving-service:8888/v1/completions"
# lltoken = "eyJhbGciOiJSUzI1NiIsInR5cCIgOiAiSldUIiwia2lkIiA6ICJ4WTdTd3k5UE1xaXRDQmNSMm5qcVl6bmoxS3NqZzV3TmdOV0xDVzdyUkhvIn0.eyJleHAiOjE3MzA4ODM3MzgsImlhdCI6MTY5OTI2MTMzOCwiYXV0aF90aW1lIjoxNjk5MjUyMDQ0LCJqdGkiOiJmN2EzMzQwYy1kNDQwLTRlMzUtYjk2ZS04YzBiMTc0Y2RhODAiLCJpc3MiOiJodHRwczovL3JlZnJhY3QtbG9naW4uZm9zZm9yLmNvbS9hdXRoL3JlYWxtcy9tb3NhaWMiLCJhdWQiOlsibW9zYWljLWdhdGVrZWVwZXIiLCJhY2NvdW50Il0sInN1YiI6IjZjMjU4MWU3LWZmMTItNDljNy04MDJmLWI2ZjQzOWQxZDIwMSIsInR5cCI6IkJlYXJlciIsImF6cCI6Im1vc2FpYy1nYXRla2VlcGVyIiwic2Vzc2lvbl9zdGF0ZSI6IjBhY2Y3YWZhLTVmMzMtNGRhZS05OGM3LTQyZDQwYTdlZTM2NiIsImFsbG93ZWQtb3JpZ2lucyI6WyIqIl0sInJlYWxtX2FjY2VzcyI6eyJyb2xlcyI6WyJNTE9QUyIsImxvbmdfbGl2ZWRfdG9rZW4iLCJzcGVjdHJhLWRldmVsb3BlciIsImRlZmF1bHQtcm9sZXMtbW9zYWljIiwicmVmcmFjdC1kZXZlbG9wZXIiLCJvZmZsaW5lX2FjY2VzcyIsImFkbWluIiwidW1hX2F1dGhvcml6YXRpb24iLCJyZWZyYWN0LWFkbWluIl19LCJyZXNvdXJjZV9hY2Nlc3MiOnsiYWNjb3VudCI6eyJyb2xlcyI6WyJtYW5hZ2UtYWNjb3VudCIsIm1hbmFnZS1hY2NvdW50LWxpbmtzIiwidmlldy1wcm9maWxlIl19fSwic2NvcGUiOiJvcGVuaWQgZW1haWwgcHJvZmlsZSIsInNpZCI6IjBhY2Y3YWZhLTVmMzMtNGRhZS05OGM3LTQyZDQwYTdlZTM2NiIsImVtYWlsX3ZlcmlmaWVkIjp0cnVlLCJuYW1lIjoiUmVmcmFjdCBCRlNJIiwicHJlZmVycmVkX3VzZXJuYW1lIjoicmVmcmFjdC5iZnNpQGZvc2Zvci5jb20iLCJnaXZlbl9uYW1lIjoiUmVmcmFjdCIsImZhbWlseV9uYW1lIjoiQkZTSSIsImVtYWlsIjoicmVmcmFjdC5iZnNpQGZvc2Zvci5jb20ifQ.b6SYLgjo9Veo3GmJ8eZjCTNupQjpfMhzsoXdYjWwRtvRnNjBfx0gOqcugO9OcGn-mm8wwpSGI5uiL30-I6SdWBjsf1ur6GztoX7j-nP_3SrJJn3UhNNqIO8LbsPi5gGRTzWtnfjz92BF1YaCXxQwPY0P_aa8vJ6JxZz5Uctn9aIPIJZZnnjC_GPXtXurmshM_tEN2kwCjhEyr7wYzRqUoMtBGfpLjZREBzgZY-x6JyYiXNtycb1d6PFcCXf7nJVV8ienEC_x7OuciDzfeqd-SQnImvAHH7rqFdi9smBN08AbkDS2uAbMrokHrmbiBpaimrR013VwCWz2KL5QYlWleA"
request_payload_file = '/data/Magicoder_payloads.txt'
report_path = '/data/vLLM_5Concurrency_1gpu.csv'

In [4]:
payload = {"prompt": "Problem: Write a function in python to check if two strings are anagrams.Input: Strings s1 and s2.Output: Boolean, indicating whether s1 and s2 are anagrams.", 
           "max_tokens": 200,
           "temperature": 0,
           "model": "/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66"}

In [5]:
def my_decorator(func):
    def calculate_execution_time(*args, **kwargs):
        start_time = time.time()
        result,_ = func(*args, **kwargs)
        end_time = time.time()
        execution_time = end_time - start_time
        data = [
            ['Payload', 'Response', 'Time (seconds)'],
            [result, _.status_code, f'{execution_time:.6f}'],
        ]
        save_report(report_path, data)
        print(f"Execution time of {func.__name__}: {execution_time:.6f} seconds")
        return result, _
    return calculate_execution_time

'{"prompt": "Problem: Write a function in python to check if two strings are anagrams.Input: Strings s1 and s2.Output: Boolean, indicating whether s1 and s2 are anagrams.", "max_tokens": 200, "temperature": 0, "model": "/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66"}'

In [6]:
def save_report(file_path, data):
    file_exists = os.path.exists(file_path)
    with open(file_path, 'a', newline='') as csv_file:
        csv_writer = csv.writer(csv_file)
        if not file_exists:
            csv_writer.writerow(data[0])
        for row in data[1:]:
            csv_writer.writerow(row)

In [22]:
! pip install aiohttp

     |████████████████████████████████| 993kB 9.7MB/s eta 0:00:01
     |████████████████████████████████| 296kB 88.3MB/s eta 0:00:01
     |████████████████████████████████| 61kB 6.2MB/s  eta 0:00:01
     |████████████████████████████████| 143kB 91.1MB/s eta 0:00:01
     |████████████████████████████████| 153kB 110.0MB/s eta 0:00:01
     |████████████████████████████████| 102kB 9.3MB/s eta 0:00:01
     |████████████████████████████████| 71kB 6.1MB/s  eta 0:00:01
ERROR: jupyterlab-server 2.22.1 has requirement jsonschema>=4.17.3, but you'll have jsonschema 3.2.0 which is incompatible.
ERROR: jupyterlab-server 2.22.1 has requirement jupyter-server<3,>=1.21, but you'll have jupyter-server 1.17.0 which is incompatible.
ERROR: jsonschema-spec 0.1.4 has requirement jsonschema<4.18.0,>=4.0.0, but you'll have jsonschema 3.2.0 which is incompatible.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [8]:
# import asyncio
# import time

# import httpx


# BASE_URL = "http://127.0.0.1:8888/v1/completions"


# async def get_mql(prompt: str, client: httpx.AsyncClient) -> str:
# #     payload = {"prompt": prompt,
# #                "max_tokens": 200,
# #                "temperature": 0,
# #                "model": "/data/Openchat_check_points/query-to-mql/jan04/merged_model"}
#     response = await client.post(f"{url}", data=payload)
#     print(".")
#     data = response.json()
#     return data


# async def main() -> None:
#     t0 = time.perf_counter()
#     # headers = {"User-Agent": "curl/7.72.0"}
#     async with httpx.AsyncClient() as client:
#         mql = await asyncio.gather(*(get_mql(prompt, client) for prompt in inps))
#     print(dict(zip(inps, mql)))
#     print(f"Total time: {time.time() - t0:.3} seconds")

In [6]:
# @my_decorator
def inference(payload_data=None):
    payload = payload_data
    headers = {
        'Content-Type': 'application/json'
    }
    start_time = time.time()
    response = requests.request("POST", url, headers=headers, json=payload)
    print("--- %s seconds ---" % (time.time() - start_time))
    return response.text

op = inference(payload)
print(op)

--- 5.233199596405029 seconds ---
{"id":"cmpl-8e7d38a59bd84dc197ba93f78a752c80","object":"text_completion","created":87829,"model":"/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66","choices":[{"index":0,"text":"\nAn anagram is a word or phrase formed by rearranging the letters of a different word or phrase, typically using all the original letters exactly once.\n\ndef are_anagrams(s1, s2):\n    return sorted(s1) == sorted(s2)\n\n# Test the function\nprint(are_anagrams(\"listen\", \"silent\"))  # True\nprint(are_anagrams(\"hello\", \"world\"))  # False","logprobs":null,"finish_reason":"stop"}],"usage":{"prompt_tokens":45,"total_tokens":148,"completion_tokens":103}}


In [7]:
import aiohttp
import asyncio
import time

start_time = time.time()
headers={"Content-type":"application/json"} 
async def main():

    async with aiohttp.ClientSession(trust_env = True, headers=headers) as session:

#         for payload in payloads:
#             url='https://refract.fosfor.com/openchatfinetunedimqlgenerator/d0721c53-c9f5-4d79-a988-2d21df46198f/score' 
        async with session.post(url, json=payload) as resp:
                mql = await resp.json(content_type=None)
                print(mql)

await main()
print("--- %s seconds ---" % (time.time() - start_time))

{'id': 'cmpl-9142a552a9a1470e8ac53e85e51b2080', 'object': 'text_completion', 'created': 87834, 'model': '/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66', 'choices': [{'index': 0, 'text': '\nAn anagram is a word or phrase formed by rearranging the letters of a different word or phrase, typically using all the original letters exactly once.\n\ndef are_anagrams(s1, s2):\n    return sorted(s1) == sorted(s2)\n\n# Test the function\nprint(are_anagrams("listen", "silent"))  # True\nprint(are_anagrams("hello", "world"))  # False', 'logprobs': None, 'finish_reason': 'stop'}], 'usage': {'prompt_tokens': 45, 'total_tokens': 148, 'completion_tokens': 103}}
--- 5.502794504165649 seconds ---
